In this notebook, the progress of the Low-Rank Multiplication program is demonstrated. It involves two matrices, where SVD is performed on matrix A while matrix B remains unchanged. The percentage determines the number of vectors to retain from the sigma matrix, which are then used to multiply matrix A by matrix B.

In [ ]:
import numpy as np

def LRM_percentage(matrix_A, matrix_B, percentage=100):
    # SVD decomposition of matrix A
    U, S, VT = np.linalg.svd(matrix_A, full_matrices=False)
    
    # Determine number of singular values to use of the S matrix
    num_singular_values = len(S)
    k = int(np.ceil(num_singular_values * (percentage / 100)))
    k = max(1, min(k, num_singular_values))  # ensure at least 1, at most all

    # Keep only top-k singular values
    S_k = S[:k]
    
    print(f"\nUsing top {k} singular values ({percentage:.1f}%) for LRM")
    print("Original Singular Values (S):", S)
    print("Top-k Singular Values (S_k):", S_k)

    # Step 1: Multiply V^T and B
    step1 = VT[:k, :] @ matrix_B  # only use top-k rows of VT

    # Step 2: Scale each row by Sigma[i,i]
    total = np.zeros_like(step1)
    for i in range(k):
        total[i, :] = S_k[i] * step1[i, :]

    # Step 3: Multiply by U
    final_result = U[:, :k] @ total  # only first k columns of U
    print("Final Result of LRM (approx):", final_result, sep="\n")
    
    return final_result

# Example matrices
matrix_A = np.array([[1, 6, 4, 5],
                     [2, -1, 0, 2],
                     [4, 3, -2, 3]])

matrix_B = np.array([[2, 5, 3],
                     [3, -2, 0],
                     [1, 3, -3],
                     [2, 5, 6]])

# Direct multiplication for comparison
direct_result = matrix_A @ matrix_B

# Test LRM with different percentages
percentages = [100, 61.7, 33.3]
for p in percentages:
    result = LRM_percentage(matrix_A, matrix_B, percentage=p)
    error_percentage = (np.linalg.norm(direct_result - result, 'fro') / np.linalg.norm(direct_result, 'fro')) * 100
    print(f"Percentage Error: {error_percentage:.6f}%")



Using top 3 singular values (100.0%) for LRM
Original Singular Values (S): [9.72158252 5.04456835 2.24569887]
Top-k Singular Values (S_k): [9.72158252 5.04456835 2.24569887]
Final Result of LRM (approx):
[[34. 30. 21.]
 [ 5. 22. 18.]
 [21. 23. 36.]]
Percentage Error: 0.000000%

Using top 2 singular values (61.7%) for LRM
Original Singular Values (S): [9.72158252 5.04456835 2.24569887]
Top-k Singular Values (S_k): [9.72158252 5.04456835]
Final Result of LRM (approx):
[[34.05349454 28.99828956 20.65750638]
 [ 5.68972038  9.08466219 13.5841323 ]
 [20.72272017 28.1921949  37.77525715]]
Percentage Error: 19.741560%

Using top 1 singular values (33.3%) for LRM
Original Singular Values (S): [9.72158252 5.04456835 2.24569887]
Top-k Singular Values (S_k): [9.72158252]
Final Result of LRM (approx):
[[35.10911005 34.6938514  32.76192158]
 [ 4.90415444  4.84614976  4.57629154]
 [18.97231815 18.74792    17.70394061]]
Percentage Error: 42.124648%
